In [1]:
import glob
from keras.models import Sequential, load_model
import numpy as np
import pandas as pd
import keras as k
from keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [3]:
df = pd.read_excel('COVID-19.xlsx')

In [4]:
df.head()

,age,sex,heart disease,lung disease,diabetes,fever,cough,shortness of breathe,pressure in the chest,close contact with corona patient,traveled country,Unnamed: 11,disease,flu,cold,corona
0,66,male,yes,no,yes,yes,yes,yes,yes,no,china,NaN,corona,0,0,1
1,56,female,no,no,yes,yes,yes,yes,no,no,china,NaN,corona,0,0,1
2,46,male,no,no,no,yes,yes,yes,yes,no,china,NaN,corona,0,0,1
3,60,male,yes,yes,yes,yes,yes,yes,yes,no,china,NaN,corona,0,0,1
4,58,male,no,no,yes,yes,yes,no,no,no,china,NaN,flu,2,0,0


In [5]:
df.drop(['Unnamed: 11'],axis=1,inplace=True)

In [6]:
df.head()

,age,sex,heart disease,lung disease,diabetes,fever,cough,shortness of breathe,pressure in the chest,close contact with corona patient,traveled country,disease,flu,cold,corona
0,66,male,yes,no,yes,yes,yes,yes,yes,no,china,corona,0,0,1
1,56,female,no,no,yes,yes,yes,yes,no,no,china,corona,0,0,1
2,46,male,no,no,no,yes,yes,yes,yes,no,china,corona,0,0,1
3,60,male,yes,yes,yes,yes,yes,yes,yes,no,china,corona,0,0,1
4,58,male,no,no,yes,yes,yes,no,no,no,china,flu,2,0,0


In [ ]:
#df['traveled country'].value_counts()

In [7]:
df['heart disease'] = df['heart disease'].map({'yes': 1, 'no': 0})
df['lung disease'] = df['lung disease'].map({'yes': 1, 'no': 0})
df['diabetes'] = df['diabetes'].map({'yes': 1, 'no': 0})
df['fever'] = df['fever'].map({'yes': 1, 'no': 0})
df['cough'] = df['cough'].map({'yes': 1, 'no': 0})
df['shortness of breathe'] = df['shortness of breathe'].map({'yes': 1, 'no': 0})
df['pressure in the chest'] = df['pressure in the chest'].map({'yes': 1, 'no': 0})
df['close contact with corona patient'] = df['close contact with corona patient'].map({'yes': 1, 'no': 0})
#df['corona patient'] = df['corona patient'].map({'corona': 1, 'flu': 2, 'cold':3})
df['sex'] = df['sex'].map({'male': 1, 'female': 0})
#df['traveled country'] = df['traveled country'].map({'china': 1, 'singapore': 2,'usa': 3, 'south korea': 4,'italy': 5, 'germany': 6,
                                                     #'switzerland': 7, 'spain': 8,'thailand': 9,'egypt':10,'belgium':11,'lebanon':12,
                                                     #'iraq':13,'iran':14,'afganistan':15,'kuwait':16,'algeria':17,'austria':18,'nepal':19,
                                                     #'malaysia':20,'sri lanka':21,'india':22,'sweden':23,'canada':24,'netherlands':25,
                                                     #'brazil':26,'greece':27,'israel':28,'russia':29,'chile':30,'mexico':31,'belarus':32,
                                                     #'cyprus':33,'turkey':34,'honduras':35,'kenya':36,'uk':37,'costa rica':38,'armenia':39,
                                                    #'ecuador':40,'bosnia':41,'azerbaijan':42,'peru':43,'panama':44,'bulgeria':45,'portugal':46})

df['traveled country'] = df['traveled country'].map({'china': 2, 'singapore': 8,'usa': 3, 'south korea': 5,'italy': 1, 'germany': 9,
                                                     'switzerland': 7, 'spain': 4,'thailand': 6,'egypt':10,'belgium':11,'lebanon':12,
                                                     'iraq':11,'iran':11,'afganistan':12,'kuwait':12,'algeria':12,'austria':12,'nepal':12,
                                                     'malaysia':10,'sri lanka':12,'india':12,'sweden':10,'canada':10,'netherlands':12,
                                                     'brazil':12,'greece':12,'israel':12,'russia':12,'chile':12,'mexico':12,'belarus':12,
                                                     'cyprus':12,'turkey':12,'honduras':12,'kenya':12,'uk':10,'costa rica':12,'armenia':12,
                                                    'ecuador':12,'bosnia':12,'azerbaijan':12,'peru':12,'panama':12,'bulgeria':12,'portugal':11})
df['disease'] = df['disease'].map({'cold': 0, 'corona': 1, 'flu': 2})

In [8]:
df.head()

,age,sex,heart disease,lung disease,diabetes,fever,cough,shortness of breathe,pressure in the chest,close contact with corona patient,traveled country,disease,flu,cold,corona
0,66,1,1,0,1,1,1,1,1,0,2,1,0,0,1
1,56,0,0,0,1,1,1,1,0,0,2,1,0,0,1
2,46,1,0,0,0,1,1,1,1,0,2,1,0,0,1
3,60,1,1,1,1,1,1,1,1,0,2,1,0,0,1
4,58,1,0,0,1,1,1,0,0,0,2,2,2,0,0


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 202 entries, 0 to 201
Data columns (total 15 columns):
age                                  202 non-null int64
sex                                  202 non-null int64
heart disease                        202 non-null int64
lung disease                         202 non-null int64
diabetes                             202 non-null int64
fever                                202 non-null int64
cough                                202 non-null int64
shortness of breathe                 202 non-null int64
pressure in the chest                202 non-null int64
close contact with corona patient    202 non-null int64
traveled country                     202 non-null int64
disease                              202 non-null int64
flu                                  202 non-null int64
cold                                 202 non-null int64
corona                               202 non-null int64
dtypes: int64(15)
memory usage: 23.8 KB


In [10]:
#Split the data
y = df['disease']
X = df.drop(['disease','flu','cold','corona'], axis=1)
#y_corona = df["corona"]
#y_flu = df['flu']
#y_cold = df['cold']


In [11]:
X.head()

,age,sex,heart disease,lung disease,diabetes,fever,cough,shortness of breathe,pressure in the chest,close contact with corona patient,traveled country
0,66,1,1,0,1,1,1,1,1,0,2
1,56,0,0,0,1,1,1,1,0,0,2
2,46,1,0,0,0,1,1,1,1,0,2
3,60,1,1,1,1,1,1,1,1,0,2
4,58,1,0,0,1,1,1,0,0,0,2


In [12]:
y.head()

0    1
1    1
2    1
3    1
4    2
Name: disease, dtype: int64

In [ ]:
#Feature Scaling
x_scaler = MinMaxScaler()
x_scaler.fit(X)
column_names = X.columns
X[column_names] = x_scaler.transform(X)

In [ ]:
X.head()

In [ ]:
#Split the data into 80% training and 20% testing 
X_train,  X_test, y_train, y_test = train_test_split(
        X, y, test_size= 0.2, shuffle=True)

In [ ]:
#Build The model
model = Sequential()
model.add(Dense(512, input_dim=len(X.columns),kernel_initializer=k.initializers.random_normal(seed=13), activation="relu"))
#model.add(Dense(128, activation='relu'))
#model.add(Dense(128, activation='relu'))
#model.add(Dense(512, activation='relu'))
model.add(Dense(3, activation="softmax"))

In [ ]:
#Compile the model
model.compile(loss='binary_crossentropy', 
                  optimizer='adam', metrics=['accuracy'])

In [ ]:
#Train the model
history = model.fit(X_train, y_train, 
                    epochs=1000, 
                    batch_size=X_train.shape[0]) 

In [ ]:
#Save the model
model.save("covid-19.model")

In [ ]:
#Visualize the models accuracy and loss
plt.plot(history.history["accuracy"])
plt.plot(history.history["loss"])
plt.title("model accuracy & loss")
plt.ylabel("accuracy and loss")
plt.xlabel("epoch")
plt.legend(['acc', 'loss'], loc='lower right')
plt.show()

In [ ]:
model

In [ ]:
model.evaluate(X_test, y_test)

In [ ]:
X_test.head()

In [ ]:
y_test.reset_index(drop=True,inplace=True)

In [ ]:
prediction = model.predict_classes(X_test)

prediction = pd.DataFrame(prediction)

prediction.columns = ['prediction']

In [ ]:
map_dict = {0:'corona',1:'flu',2:'cold'} 
#y_test["test"] = prediction.loc[:,0].map(map_dict)
y_test['predicted'] = prediction


y_test.head()

In [ ]:
map_dict = {0:'corona',1:'flu',2:'cold'} 
y_test["predicted"] = y_test["predicted"].map(map_dict)
y_test.head()

In [ ]:
y_test.to_excel("output.xlsx")

In [13]:
import sklearn

In [14]:
X.shape


(202, 11)

In [15]:
y.shape

(202,)

In [16]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=.2)
print(x_train.shape,y_train.shape, x_test.shape,y_test.shape)

(161, 11) (161,) (41, 11) (41,)


In [17]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5) # k-classifier= 5
knn.fit(x_train,y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='uniform')

In [18]:
y_pred= knn.predict(x_test)
y_pred=np.array(y_pred)
y_test=np.array(y_test)
for i in range(len(y_pred)):
  print(y_pred[i],y_test[i])

1 1
1 1
0 0
1 1
1 1
1 0
1 1
0 0
1 2
1 1
2 2
0 0
2 1
1 2
1 1
1 1
1 1
1 1
1 2
1 2
1 2
1 1
2 1
2 2
0 0
1 1
1 2
1 1
1 1
2 1
2 2
1 0
1 1
1 1
2 1
1 1
1 0
0 2
1 2
2 0
1 1


In [19]:
from sklearn import metrics
print(metrics.classification_report(y_test,y_pred))
print(metrics.accuracy_score(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.80      0.50      0.62         8
           1       0.64      0.82      0.72        22
           2       0.38      0.27      0.32        11

   micro avg       0.61      0.61      0.61        41
   macro avg       0.61      0.53      0.55        41
weighted avg       0.60      0.61      0.59        41

0.6097560975609756


In [20]:
from sklearn.tree import DecisionTreeClassifier
dtc= DecisionTreeClassifier()
dtc.fit(x_train,y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [21]:
y_pred=dtc.predict(x_test)
y_pred=np.array(y_pred)
y_test=np.array(y_test)
print(metrics.accuracy_score(y_test,y_pred))

0.8536585365853658


In [22]:
import pickle
pickle.dump(dtc, open('dtc.dump','wb'))

In [23]:
loaded_model=pickle.load(open('dtc.dump', 'rb'))

In [24]:
dotfile = open("tree.dot","w")
sklearn.tree.export_graphviz(dtc, out_file=dotfile, feature_names=X.columns)
dotfile.close()


In [25]:
X.columns

Index(['age', 'sex', 'heart disease', 'lung disease', 'diabetes', 'fever',
       'cough', 'shortness of breathe', 'pressure in the chest',
       'close contact with corona patient', 'traveled country'],
      dtype='object')

In [30]:
from sklearn.tree import _tree

def tree_to_code(tree, feature_names):
    tree_ = tree.tree_
    feature_name = [
        feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]
    print ("def tree({}):".format(", ".join(feature_names)))

    def recurse(node, depth):
        indent = "  " * depth
        if tree_.feature[node] != _tree.TREE_UNDEFINED:
            name = feature_name[node]
            threshold = tree_.threshold[node]
            print ("{}if ({} <= {})".format(indent, name, threshold))
            print("{")
            recurse(tree_.children_left[node], depth + 1)
            print ("{}else  # if {} > {}".format(indent, name, threshold))
            print("{")
            recurse(tree_.children_right[node], depth + 1)
        else:
            print ("{}return {};".format(indent, tree_.value[node]))
            print("}")

    recurse(0, 1)

In [31]:
tree_to_code(dtc,X.columns)

def tree(age, sex, heart disease, lung disease, diabetes, fever, cough, shortness of breathe, pressure in the chest, close contact with corona patient, traveled country):
  if (shortness of breathe <= 0.5)
{
    if (fever <= 0.5)
{
      if (pressure in the chest <= 0.5)
{
        if (age <= 34.0)
{
          if (age <= 31.0)
{
            return [[4. 0. 0.]];
}
          else  # if age > 31.0
{
            return [[0. 0. 1.]];
}
        else  # if age > 34.0
{
          return [[18.  0.  0.]];
}
      else  # if pressure in the chest > 0.5
{
        if (traveled country <= 7.5)
{
          return [[0. 2. 0.]];
}
        else  # if traveled country > 7.5
{
          if (traveled country <= 11.0)
{
            return [[1. 0. 0.]];
}
          else  # if traveled country > 11.0
{
            return [[0. 0. 2.]];
}
    else  # if fever > 0.5
{
      if (close contact with corona patient <= 0.5)
{
        if (pressure in the chest <= 0.5)
{
          return [[ 0.  0. 39.]];
}
        else 